Импортируем библиотеки. Загружаем данные.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Удаляем ненужный столбец.

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Rain.csv', sep=',')
data=data.drop(['Unnamed: 0'], axis='columns')

Создаем обучающую и тестовую модели.

In [ ]:
X = data[data.columns[:-1]]
y = data['RainTomorrow']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

Проводим нормализацию.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

In [ ]:
#X_train_scaled.reset_index(inplace=True, drop=True)
#y_train.reset_index(drop=True, inplace=True)
#RainYes = X_train_scaled[y_train == 1]
#X_train_strat = pd.concat([X_train_scaled, RainYes, RainYes,RainYes], ignore_index=True)
#y_train_strat = pd.concat([y_train, pd.Series(np.ones(X_train_strat.shape[0]-X_train_scaled.shape[0]))],
                          #ignore_index=True)

Заготовка таблицы. Колонки - признаки, строчки - глубина регуляризации, ячейки - веса.

In [ ]:
col = ['coef_%s'%i for i in  list(X_train_scaled.columns.values) ]
ind = ['C=%f'%i for i in [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4, 10**5]]
coef_matrix_simple = pd.DataFrame(index=ind, columns=col)

Добавляем библиотеки метрик.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, roc_auc_score, f1_score, fbeta_score, auc

Обучаем логистическую регрессию L2-регуляризаций, считаем recall и оцениваем веса признаков.

In [ ]:
accuracy = []
C_list = [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4, 10**5]

for i in range(0, len(C_list)):
    clf = LogisticRegression(C=C_list[i], random_state=10, solver='lbfgs', max_iter=200)
    clf.fit(X_train_scaled, y_train)
    a=clf.predict(X_test_scaled)
    #pr, rec, _ = precision_recall_curve(y_test, a)
    current_score = recall_score(y_test, a)
    coef_matrix_simple.iloc[i,:]=clf.coef_
    accuracy.append(current_score)
    print('При С =', C_list[i], 'точность равна', current_score)
max_score = 0    
for i in range(0, len(accuracy)):
    if max_score < accuracy[i]:
        max_score = accuracy[i]
        C_for_max_score = i
print('\n')
print('Максимальная точность (score), равная', max_score,', получается при С, равном', C_list[C_for_max_score])
pd.options.display.float_format = '{:,.3g}'.format
coef_matrix_simple

При С = 1e-05 точность равна 0.0
При С = 0.0001 точность равна 0.234375
При С = 0.001 точность равна 0.6953125
При С = 0.01 точность равна 0.7421875
При С = 0.1 точность равна 0.7682291666666666
При С = 1 точность равна 0.7630208333333334
При С = 10 точность равна 0.7734375
При С = 100 точность равна 0.7682291666666666
При С = 1000 точность равна 0.7682291666666666
При С = 10000 точность равна 0.7682291666666666
При С = 100000 точность равна 0.7682291666666666


Максимальная точность (score), равная 0.7734375 , получается при С, равном 10


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,coef_MinTemp,coef_MaxTemp,coef_Rainfall,coef_Evaporation,coef_Sunshine,coef_WindGustSpeed,coef_WindSpeed9am,coef_WindSpeed3pm,coef_Humidity9am,coef_Humidity3pm,coef_Pressure9am,coef_Pressure3pm,coef_Cloud9am,coef_Cloud3pm,coef_Temp9am,coef_Temp3pm,coef_RainToday
C=0.000010,0.000207,-0.000394,0.000158,-0.000157,-0.00164,0.000384,0.000139,0.000173,0.00076,0.0012,-0.000484,-0.000445,0.00153,0.00165,-6.87e-05,-0.000442,0.00193
C=0.000100,0.00207,-0.00385,0.00155,-0.00154,-0.0162,0.0038,0.00137,0.00171,0.00747,0.0119,-0.0048,-0.00442,0.015,0.0162,-0.000646,-0.00433,0.0189
C=0.001000,0.02,-0.0309,0.0133,-0.0127,-0.14,0.0353,0.0123,0.0153,0.0636,0.103,-0.0442,-0.0416,0.126,0.14,-0.00326,-0.0356,0.158
C=0.010000,0.139,-0.0872,0.0632,-0.0481,-0.679,0.246,0.0623,0.0844,0.287,0.531,-0.289,-0.301,0.473,0.64,0.0395,-0.128,0.606
C=0.100000,0.274,0.0443,0.218,-0.157,-1.62,1.19,-0.0183,0.085,0.772,1.54,-0.949,-1.3,0.297,1.09,0.212,-0.147,0.785
C=1.000000,-0.375,0.645,1.11,-0.55,-2.33,3.6,-0.57,-0.674,1.31,2.92,-0.184,-3.44,-0.13,0.959,0.402,0.12,0.671
C=10.000000,-0.806,1.16,3.71,-0.661,-2.32,5.27,-0.836,-1.37,1.45,3.57,5.35,-9.17,-0.118,0.904,1.11,-0.907,0.597
C=100.000000,-0.689,2.05,5.29,-0.495,-2.23,5.66,-0.819,-1.59,1.63,3.44,9.91,-13.8,-0.0635,0.914,1.77,-3.08,0.586
C=1000.000000,-0.657,2.31,5.54,-0.454,-2.22,5.71,-0.812,-1.62,1.68,3.39,10.8,-14.7,-0.0519,0.915,1.9,-3.62,0.587
C=10000.000000,-0.65,2.34,5.57,-0.452,-2.21,5.72,-0.811,-1.62,1.68,3.38,10.9,-14.8,-0.0509,0.916,1.91,-3.69,0.587


При С=10 recall максимальный. На маленькой глубине регуляризации наблюдается переобучение.

Оценим остальные метрики.

In [ ]:
clf = LogisticRegression(C=C_list[C_for_max_score], random_state=10,max_iter=200,solver='lbfgs')
clf.fit(X_train_scaled, y_train)
a=clf.predict(X_test_scaled)
print('Accuracy:', accuracy_score(y_test, a))
print('Precision:', precision_score(y_test, a))
print('Recall:', recall_score(y_test, a))
print('ROC AUC:', roc_auc_score(y_test, a))
pr, rec, _ = precision_recall_curve(y_test, a)
print('PRC AUC:', auc(rec, pr))
print('F1:', f1_score(y_test, a))
print('F0.5:', fbeta_score(y_test, a, 0.5))
print('F2:', fbeta_score(y_test, a, 2))

Accuracy: 0.7847593582887701
Precision: 0.8005390835579514
Recall: 0.7734375
ROC AUC: 0.7850703983516484
PRC AUC: 0.8451433719928795
F1: 0.7867549668874173
F0.5: 0.7949678800856532
F2: 0.7787100157315155


Обучаем логистическую регрессию L1-регуляризаций, считаем recall и оцениваем веса признаков.

In [ ]:
 accuracy = []
C_list = [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4, 10**5]

for i in range(0, len(C_list)):
    clf = LogisticRegression(C=C_list[i], random_state=10, penalty='l1',solver='liblinear', max_iter=200)
    clf.fit(X_train_scaled, y_train)
    a=clf.predict(X_test_scaled)
    #pr, rec, _ = precision_recall_curve(y_test, a)
    current_score = recall_score(y_test, a)
    coef_matrix_simple.iloc[i,:]=clf.coef_
    accuracy.append(current_score)
    print('При С =', C_list[i], 'точность равна', current_score)
max_score = 0    
for i in range(0, len(accuracy)):
    if max_score < accuracy[i]:
        max_score = accuracy[i]
        C_for_max_score = i
print('\n')
print('Максимальная точность (score), равная', max_score,', получается при С, равном', C_list[C_for_max_score])
coef_matrix_simple

При С = 1e-05 точность равна 0.0
При С = 0.0001 точность равна 0.0
При С = 0.001 точность равна 0.0
При С = 0.01 точность равна 0.6979166666666666
При С = 0.1 точность равна 0.7708333333333334
При С = 1 точность равна 0.765625
При С = 10 точность равна 0.7708333333333334
При С = 100 точность равна 0.7682291666666666
При С = 1000 точность равна 0.7682291666666666
При С = 10000 точность равна 0.7682291666666666
При С = 100000 точность равна 0.7682291666666666


Максимальная точность (score), равная 0.7708333333333334 , получается при С, равном 0.1


,coef_MinTemp,coef_MaxTemp,coef_Rainfall,coef_Evaporation,coef_Sunshine,coef_WindGustSpeed,coef_WindSpeed9am,coef_WindSpeed3pm,coef_Humidity9am,coef_Humidity3pm,coef_Pressure9am,coef_Pressure3pm,coef_Cloud9am,coef_Cloud3pm,coef_Temp9am,coef_Temp3pm,coef_RainToday
C=0.000010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C=0.000100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C=0.001000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C=0.010000,0,0,0,0,-0.899,0,0,0,0,0,0,0,0,0.395,0,0,0.535
C=0.100000,0,0,0,0,-2.07,1.39,0,0,0.085,2.53,0,-2.84,0,0.878,0,0,0.782
C=1.000000,0,0.151,1.41,-0.0375,-2.35,4.48,-0.795,-1.09,1.06,3.43,3.74,-7.94,-0.115,0.834,0,0,0.686
C=10.000000,-0.539,1.51,5.08,-0.406,-2.22,5.59,-0.836,-1.55,1.49,3.55,9.81,-13.8,-0.0649,0.918,1.4,-2.35,0.596
C=100.000000,-0.649,2.2,5.52,-0.451,-2.22,5.7,-0.815,-1.61,1.66,3.41,10.7,-14.6,-0.0534,0.916,1.86,-3.46,0.586
C=1000.000000,-0.661,2.27,5.57,-0.456,-2.22,5.71,-0.813,-1.62,1.68,3.4,10.8,-14.7,-0.0522,0.916,1.9,-3.57,0.585
C=10000.000000,-0.662,2.28,5.57,-0.456,-2.22,5.72,-0.813,-1.62,1.68,3.39,10.8,-14.7,-0.0521,0.916,1.91,-3.58,0.585


При С=0,1 и С=10 recall максимальный, однако при С=0,1 больше зануленных признаков. Это поможет определить значимость признаков. Значимыми оказались: sunshine(количество часов солнечной погоды), Humidity3pm(влажность в 3 часа дня), WindGustSpeed (скорость сильнейшего порыва ветра за день), Pressure3pm(давление в 3 часа дня). 

Смотрим остальные метрики.

In [ ]:
clf = LogisticRegression(C=C_list[C_for_max_score], random_state=10,max_iter=200,penalty='l1',solver='liblinear')
clf.fit(X_train_scaled, y_train)
a=clf.predict(X_test_scaled)
print('Accuracy:', accuracy_score(y_test, a))
print('Precision:', precision_score(y_test, a))
print('Recall:', recall_score(y_test, a))
print('ROC AUC:', roc_auc_score(y_test, a))
pr, rec, _ = precision_recall_curve(y_test, a)
print('PRC AUC:', auc(rec, pr))
print('F1:', f1_score(y_test, a))
print('F0.5:', fbeta_score(y_test, a, 0.5))
print('F2:', fbeta_score(y_test, a, 2))

Accuracy: 0.7780748663101604
Precision: 0.7914438502673797
Recall: 0.7708333333333334
ROC AUC: 0.7782738095238095
PRC AUC: 0.8399621212121213
F1: 0.7810026385224274
F0.5: 0.7872340425531915
F2: 0.774869109947644


Выкидываем зануленные признаки и оценим без них.

In [ ]:
X_train_scaled=X_train_scaled.drop(['Rainfall','Cloud9am','MinTemp','MaxTemp','Rainfall','Evaporation','WindSpeed9am','WindSpeed3pm','Pressure9am','Temp9am','Temp3pm'],axis='columns')

In [ ]:
X_test_scaled=X_test_scaled.drop(['Rainfall','Cloud9am','MinTemp','MaxTemp','Rainfall','Evaporation','WindSpeed9am','WindSpeed3pm','Pressure9am','Temp9am','Temp3pm'],axis='columns')

In [ ]:
clf = LogisticRegression(C=C_list[C_for_max_score], random_state=10,max_iter=200,penalty='l1',solver='liblinear')
clf.fit(X_train_scaled, y_train)
a=clf.predict(X_test_scaled)
print('Accuracy:', accuracy_score(y_test, a))
print('Precision:', precision_score(y_test, a))
print('Recall:', recall_score(y_test, a))
print('ROC AUC:', roc_auc_score(y_test, a))
pr, rec, _ = precision_recall_curve(y_test, a)
print('PRC AUC:', auc(rec, pr))
print('F1:', f1_score(y_test, a))
print('F0.5:', fbeta_score(y_test, a, 0.5))
print('F2:', fbeta_score(y_test, a, 2))

Accuracy: 0.7780748663101604
Precision: 0.7914438502673797
Recall: 0.7708333333333334
ROC AUC: 0.7782738095238095
PRC AUC: 0.8399621212121213
F1: 0.7810026385224274
F0.5: 0.7872340425531915
F2: 0.774869109947644
